In [1]:
import numpy
from scipy import linalg, special
from scipy.integrate import solve_ivp
from matplotlib import pyplot
import matplotlib

#matplotlib.rc("font", **{"family":  "serif", "weight": "normal", "size": 20})

pyplot.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
    "font.size": "28",
    "text.latex.preamble": "\\usepackage[utf8]{inputenc}\\usepackage[T1]{fontenc}\\usepackage{lmodern}"
})

from tqdm.notebook import tqdm

import joblib

from born_markov import *

In [2]:
omega = 0.2 # eV
lamda = 0.4 # eV
e_0 = 0.3 + (lamda**2/omega) # eV
N = 10
Gamma = 0.01 # eV
T = 100 # K
voltage = .2


#solver = create_holstein_solver(e_0, omega, lamda, N, Gamma, -0.5*voltage, 0.5*voltage, T, T, use_exp=False)


#rho_ss, L = solver.find_steady_state()
#print(1e3*numpy.array(solver.get_current(rho_ss)))


solver2 = create_holstein_solver_via_diagonalization(e_0, omega, lamda, N, Gamma, -0.5*voltage, 0.5*voltage, T, T)


rho_ss, L = solver2.find_steady_state()
print(1e3*numpy.array(solver2.get_current(rho_ss)))

[-0.53968552+0.j  0.53968552+0.j]


In [ ]:

omega = 0.2 # eV
lamda = 0.4 # eV
e_0 = 0.3 + (lamda**2/omega) # eV
N = 10
Gamma = 0.01 # eV
T = 100 # K
voltages = numpy.linspace(.0, 2., 101)

def my_func(i, voltage):
    #solver = create_holstein_solver(e_0, omega, lamda, N, Gamma, -0.5*voltage, 0.5*voltage, T, T)
    solver = test_diag(e_0, omega, lamda, N, Gamma, -0.5*voltage, 0.5*voltage, T, T)
    rho_0 = numpy.zeros((2*N, 2*N))
    rho_0[0,0] = 1
    #rho_ss = solver.propagate_RK(rho_0, 400, dt_max=1e-2)
    rho_ss, L = solver.find_steady_state()
    return (solver.get_current(rho_ss), numpy.diag(rho_ss)[::2] + numpy.diag(rho_ss)[1::2])

results = joblib.Parallel(n_jobs=6)(joblib.delayed(my_func)(i, voltage) for i, voltage in enumerate(tqdm(voltages)))

currents = numpy.zeros((len(voltages), 2))
occupations = numpy.zeros((len(voltages), N))

for i, voltage in enumerate(voltages):
    currents[i] = results[i][0]
    occupations[i] = results[i][1]

#for i, voltage in enumerate(tqdm(voltages)):
    
    
pyplot.figure(figsize=(15,10))
pyplot.plot(voltages, currents[:,0], label="$J_L$")
pyplot.plot(voltages, currents[:,1], label="$J_R$")

pyplot.grid()
pyplot.legend(loc=1)
pyplot.xlabel("$\\Delta\\phi$ [V]")
pyplot.ylabel("$J_K$ [mA]")
pyplot.show()

pyplot.figure(figsize=(15,10))
for i in range(N):
    pyplot.plot(voltages, occupations[:,i], label="$N="+str(i)+"$")

pyplot.grid()
pyplot.legend(loc=1)
pyplot.xlabel("$\\Delta\\phi$ [V]")
pyplot.ylabel("$N$ [1]")
pyplot.show()

#numpy.save("HOLc_0.4_100K", currents)
#numpy.save("HOLo_0.4_100K", occupations)

In [ ]:
fig, (ax1, ax2) = pyplot.subplots(1, 2, figsize=(20,10), tight_layout=True)

voltages = numpy.linspace(0., 2., 501)

ax1.plot(voltages, 1e3*numpy.load("HOLc_0.12_100K.npy")[:,0], label="$T$=100K", c="tab:blue")
ax1.plot(voltages, 1e3*numpy.load("HOLc_0.12_100K.npy")[:,1], c="tab:blue", ls="--")
ax1.plot(voltages, 1e3*numpy.load("HOLc_0.12_200K.npy")[:,0], label="$T$=200K", c="tab:orange")
ax1.plot(voltages, 1e3*numpy.load("HOLc_0.12_200K.npy")[:,1], c="tab:orange", ls="--")
ax1.plot(voltages, 1e3*numpy.load("HOLc_0.12_300K.npy")[:,0], label="$T$=300K", c="tab:green")
ax1.plot(voltages, 1e3*numpy.load("HOLc_0.12_300K.npy")[:,1], c="tab:green", ls="--")
ax1.grid()
ax1.legend(loc=5, framealpha=1.)
ax1.set_xlabel("$\\Delta\\phi$ [V]")
ax1.set_ylabel("$J_K$ [µA]")
ax1.set_ylim(-1.3, 1.3)
ax1.text(-.7, -.035, "\\textbf{(a)}")

voltages = numpy.linspace(0., 5., 501)

ax2.plot(voltages, 1e3*numpy.load("HOLc_0.4_100K.npy")[:,0], label="$T$=100K", c="tab:blue") 
ax2.plot(voltages, 1e3*numpy.load("HOLc_0.4_100K.npy")[:,1], c="tab:blue", ls="--")
ax2.plot(voltages, 1e3*numpy.load("HOLc_0.4_200K.npy")[:,0], label="$T$=200K", c="tab:orange")
ax2.plot(voltages, 1e3*numpy.load("HOLc_0.4_200K.npy")[:,1], c="tab:orange", ls="--")
ax2.plot(voltages, 1e3*numpy.load("HOLc_0.4_300K.npy")[:,0], label="$T$=300K", c="tab:green")
ax2.plot(voltages, 1e3*numpy.load("HOLc_0.4_300K.npy")[:,1], c="tab:green", ls="--")
ax2.grid()
ax2.legend(loc=5, framealpha=1.)
ax2.set_xlabel("$\\Delta\\phi$ [V]")
#ax2.set_ylabel("$J_K$ [µA]")
ax2.set_ylim(-1.3, 1.3)
ax2.text(-1.25, -.035, "\\textbf{(b)}")

pyplot.savefig("HOL_currents.pdf")
pyplot.show()

In [ ]:
def FC_plus(N, lamda, omega):
    def fc_plus(n, m):
        mu = lamda / omega
        summe = 0
        mini = numpy.minimum(n, m)
        abso = numpy.abs(n-m)
        for i in range(mini+1):
            summe += (-1 * (mu**2))**i / (numpy.math.factorial(abso + i) * numpy.math.factorial(mini - i) * numpy.math.factorial(i))
        
        if n > m:
            alt = mu**abso
        elif n == m:
            alt = 1
        else:
            alt = (-mu)**abso
        
        return numpy.exp(-0.5 * (mu**2)) * summe * numpy.sqrt(float(numpy.math.factorial(n) * numpy.math.factorial(m))) * alt
    
    FC_plus = numpy.zeros((N, N), dtype=numpy.float64)
    for n in range(N):
        for m in range(N):
            FC_plus[n,m] = fc_plus(n,m)
    return FC_plus

def FC_minus(N, lamda, omega):
    def fc_minus(n, m):
        mu = -lamda / omega
        summe = 0
        mini = numpy.minimum(n, m)
        abso = numpy.abs(n-m)
        for i in range(mini+1):
            summe += (-1 * (mu**2))**i / (numpy.math.factorial(abso + i) * numpy.math.factorial(mini - i) * numpy.math.factorial(i))
        if n > m:
            alt = mu**abso
        elif n == m:
            alt = 1
        else:
            alt = (-mu)**abso
        return numpy.exp(-0.5 * (mu**2)) * summe * numpy.sqrt(float(numpy.math.factorial(n) * numpy.math.factorial(m))) * alt
    
    FC_minus = numpy.zeros((N, N), dtype=numpy.float64)
    for n in range(N):
        for m in range(N):
            FC_minus[n,m] = fc_minus(n,m)
    return FC_minus

fig, (ax1, ax2) = pyplot.subplots(1, 2, figsize=(20,10), tight_layout=True)

ax1.imshow(abs(FC_plus(15, .12, .2)), origin="lower")
ax1.text(-1.4, 4.85, "\\textbf{(a)}")

ax2.imshow(abs(FC_plus(15, .4, .2)), origin="lower")
ax2.text(-1.4, 4.85, "\\textbf{(b)}")

#pyplot.savefig("HOL_franckcondon.pdf")
pyplot.show()